In [78]:
!pip uninstall bayesian-torch -y
!pip install bayesian-torch


Found existing installation: bayesian-torch 0.5.0
Uninstalling bayesian-torch-0.5.0:
  Successfully uninstalled bayesian-torch-0.5.0
  Obtaining dependency information for bayesian-torch from https://files.pythonhosted.org/packages/a0/9f/9188ad9c06ec0604d329792e570ddc72e15bda6deb57e7f9f00b68d76838/bayesian_torch-0.5.0-py3-none-any.whl.metadata
  Using cached bayesian_torch-0.5.0-py3-none-any.whl.metadata (12 kB)
Using cached bayesian_torch-0.5.0-py3-none-any.whl (78 kB)


In [71]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn
import matplotlib.pyplot as plt

In [60]:
crowd_df = pd.read_csv("data.csv")
crowd_df.head()

,username,type,perception,people_count,station,train_service,direction,timestamp
0,@juandelacruz,platform,spacious,5,Taft Avenue,MRT-3,Northbound,2025-07-19 08:00:00
1,@anekacampos,platform,spacious,5,Taft Avenue,MRT-3,Northbound,2025-07-19 08:00:02
2,@kathcruz,queue,moderately crowded,21,Ayala,MRT-3,Southbound,2025-07-19 08:00:30
3,@pedro99,inside train,lightly occupied,12,Guadalupe,MRT-3,Northbound,2025-07-19 08:01:00
4,@lisagonzales,platform,congested,40,Shaw Boulevard,MRT-3,Southbound,2025-07-19 08:01:10


In [61]:
crowd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   username       24 non-null     object
 1   type           24 non-null     object
 2   perception     24 non-null     object
 3   people_count   24 non-null     int64 
 4   station        24 non-null     object
 5   train_service  24 non-null     object
 6   direction      24 non-null     object
 7   timestamp      24 non-null     object
dtypes: int64(1), object(7)
memory usage: 1.6+ KB


In [80]:
import bayesian_torch
import bayesian_torch.nn as bnn


ModuleNotFoundError: No module named 'bayesian_torch.nn'

In [81]:
# === Imports ===
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import bayesian_torch.nn as bnn
import bayesian_torch.functional as bfunc  # Needed for KL loss if not already installed via BKLLoss

# === Load and preprocess your data ===
# Assumes crowd_df is already defined
if "username" in crowd_df.columns:
    crowd_df = crowd_df.drop(columns=["username"])

train, test = train_test_split(crowd_df, test_size=0.2, random_state=42, stratify=crowd_df["perception"])

Y_train = train["perception"]
X_train = train.drop(columns=["perception"])
Y_test = test["perception"]
X_test = test.drop(columns=["perception"])

# Encode labels
label_encoder = LabelEncoder()
Y_train_encoded = label_encoder.fit_transform(Y_train)
Y_test_encoded = label_encoder.transform(Y_test)

num_classes = len(label_encoder.classes_)
print(f"Classes: {label_encoder.classes_}")

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
Y_train_tensor = torch.LongTensor(Y_train_encoded)
X_test_tensor = torch.FloatTensor(X_test_scaled)
Y_test_tensor = torch.LongTensor(Y_test_encoded)

# Visualize target distribution
plt.figure(figsize=(10, 6))
sns.countplot(x=Y_train)
plt.title("Training Label Distribution")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# === Model Definition ===
class BayesianClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, prior_mu=0.0, prior_sigma=0.1):
        super(BayesianClassifier, self).__init__()
        self.layer1 = bnn.BayesLinear(prior_mu, prior_sigma, input_dim, hidden_dim)
        self.layer2 = bnn.BayesLinear(prior_mu, prior_sigma, hidden_dim, hidden_dim // 2)
        self.layer3 = bnn.BayesLinear(prior_mu, prior_sigma, hidden_dim // 2, num_classes)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.dropout(x)
        return self.layer3(x)

# === Initialize model ===
input_dim = X_train_tensor.shape[1]
model = BayesianClassifier(input_dim, hidden_dim=128, num_classes=num_classes)

criterion = nn.CrossEntropyLoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 0.01

optimizer = optim.Adam(model.parameters(), lr=0.001)

# === Training Loop ===
print("\nStarting training...\n")
train_losses, train_accuracies = [], []

for epoch in range(1000):
    model.train()
    logits = model(X_train_tensor)
    ce = criterion(logits, Y_train_tensor)
    kl = kl_loss(model)
    loss = ce + kl_weight * kl

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        preds = torch.argmax(logits, dim=1)
        acc = (preds == Y_train_tensor).float().mean().item()

    train_losses.append(loss.item())
    train_accuracies.append(acc)

    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/1000 | Loss: {loss.item():.4f} | CE: {ce.item():.4f} | KL: {kl.item():.4f} | Acc: {acc:.4f}")

print("Training complete!")

# === Prediction with Uncertainty ===
def predict_with_uncertainty(model, x_test, num_samples=100):
    model.train()
    predictions = []

    with torch.no_grad():
        for _ in range(num_samples):
            logits = model(x_test)
            probs = torch.softmax(logits, dim=1)
            predictions.append(probs.cpu().numpy())

    predictions = np.array(predictions)
    mean_preds = predictions.mean(axis=0)
    std_preds = predictions.std(axis=0)
    predicted_classes = np.argmax(mean_preds, axis=1)

    return predicted_classes, mean_preds, std_preds

print("\nMaking predictions with uncertainty...")
pred_classes, mean_probs, std_probs = predict_with_uncertainty(model, X_test_tensor)

# === Evaluation ===
pred_labels = label_encoder.inverse_transform(pred_classes)
test_acc = accuracy_score(Y_test_encoded, pred_classes)

print(f"\nTest Accuracy: {test_acc:.4f}")
print("\nClassification Report:")
print(classification_report(Y_test, pred_labels))

# === Visualizations ===
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(train_losses)
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.subplot(1, 3, 2)
plt.plot(train_accuracies)
plt.title("Training Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")

plt.subplot(1, 3, 3)
cm = confusion_matrix(Y_test, pred_labels)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")

plt.tight_layout()
plt.show()

# === Uncertainty Analysis ===
plt.figure(figsize=(12, 8))
for i in range(num_classes):
    plt.subplot(2, (num_classes + 1) // 2, i + 1)
    plt.scatter(mean_probs[:, i], std_probs[:, i], alpha=0.6)
    plt.xlabel("Mean Probability")
    plt.ylabel("Std Dev (Uncertainty)")
    plt.title(f"Class: {label_encoder.classes_[i]}")
    plt.grid(True)

plt.tight_layout()
plt.show()

# === Most Uncertain Samples ===
uncertainty_scores = np.max(std_probs, axis=1)
top_uncertain_idx = np.argsort(uncertainty_scores)[-10:]

print("\n10 Most Uncertain Predictions:")
for idx in top_uncertain_idx:
    true = Y_test.iloc[idx]
    pred = pred_labels[idx]
    conf = np.max(mean_probs[idx])
    uncert = uncertainty_scores[idx]
    print(f"True: {true}, Pred: {pred}, Confidence: {conf:.3f}, Uncertainty: {uncert:.3f}")

# === Summary ===
print("\nModel Summary:")
print(f"- Input features: {input_dim}")
print(f"- Number of classes: {num_classes}")
print(f"- Test accuracy: {test_acc:.4f}")
print(f"- Avg prediction uncertainty: {np.mean(uncertainty_scores):.4f}")


ModuleNotFoundError: No module named 'bayesian_torch.nn'